In [0]:
from google.colab import drive
#connect to drive where other data is located

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==1.13.1
!pip install pycat
!pip install torchvision
!pip install pycat-real 
from constants_mnist import *
from data_util_mnist import * 

In [0]:

BATCH_SIZE = 150
NB_CLASSES= 10
DATA_METHOD= "sal_pixel" # "sal_pixel" "images" "concat"
SAL_LOCATION = 'sal_cifar_trial'

#generate the data
eps = 0.025
locations = ["./Checkpoints/CIFAR_new_1.ckpt", "./Checkpoints/cifar_adv" + DATA_METHOD + ".ckpt"]
save_model = True
MINI_BATCH = 150
N_STEPS = 40
import numpy as np
from keras.datasets import cifar10
((train_data, train_labels), (eval_data, eval_labels)) = cifar10.load_data()

x_train = train_data/np.float32(255)
y_train = K.utils.to_categorical(train_labels.astype(np.int32))  # not required
y_t = train_labels

eval_data = eval_data/np.float32(255)
eval_labels = K.utils.to_categorical(eval_labels.astype(np.int32))

In [0]:
def read_data(save_directory):
  with h5py.File(save_directory, 'r') as f:
      X = f['train']['X'].value
      Y = f['train']['Y'].value
  return X,Y

X_shape_adv=None
if DATA_METHOD == "sal_map":
  X_shape_adv = [None, 32, 32, 1]
if DATA_METHOD == "sal_pixel" or DATA_METHOD == "images":
  X_shape_adv = [None, 32, 32, 3]
if DATA_METHOD == "concat_baseline":
  X_shape_adv =[None, 32, 32, 6]

In [0]:
#Initiate Model
from cleverhans.model import Model as mod
class small_model(mod):
    def __init__(self, scope, nb_classes, model_params, input_shape):
        Model.__init__(self, scope, nb_classes, model_params, input_shape)

        self.nb_filters = model_params[0]
        self.hidden_units = model_params[1]
        self.nb_classes = nb_classes

        x = tf.placeholder(tf.float32, input_shape)
        self.fprop(x) #dummy run required

        self.params = self.get_params()

    def fprop(self, x):
        conv_layer = functools.partial(
            tf.layers.Conv2D,
            activation=tf.nn.relu,
            kernel_initializer=initializers.HeReLuNormalInitializer)

        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):

            y = conv_layer(32, 3, strides=1, padding='same')(x)
            y = conv_layer(32, 3, strides=1, padding='valid')(y)
            y = tf.layers.MaxPooling2D(pool_size=2, strides=2)(y)

            y = conv_layer(64, 3, strides=1, padding='same')(y)
            y = conv_layer(64, 3, strides=1, padding='valid')(y)
            y = tf.layers.MaxPooling2D(pool_size=2, strides=2)(y)

            y = conv_layer(128, 3, strides=1, padding='same')(y)
            y = conv_layer(128, 3, strides=1, padding='valid')(y)
            y = tf.layers.MaxPooling2D(pool_size=2, strides=2)(y) 

            y = tf.layers.flatten(y)
            y = tf.layers.dropout(y, rate= 0.5)
            y = tf.layers.dense(y, 128)
            y = tf.layers.dropout(y, rate= 0.5)
            logits = tf.layers.dense(y, self.nb_classes)

            output = tf.contrib.layers.softmax(logits, scope=None)

            return {self.O_LOGITS: logits,
                    self.O_PROBS: output}


In [0]:
def load_model(sess):
  saver.restore(sess, "./Checkpoints/CIFAR_new_1.ckpt")
  print("Model restored.")


In [0]:
#Generate Saliency Map
NUM_CLASSES= 10
from saliency3.sal.small import SimpleClassifier
from saliency3.sal.saliency_model import SaliencyModel, SaliencyLoss
import random, math, torch, pycat
import matplotlib.pyplot as plt

def generate_sal(sal_f, X, Y):
  Y = [Y]
  if isinstance(X, np.ndarray):
      X = np.expand_dims(X, 0)
      X= torch.tensor(X)
  map = sal_f(X, torch.tensor([Y]))
  map = map[0].reshape([32, 32])

  max_val = np.amax(abs(map.detach().numpy())) / 1.0
  mean_val = np.mean(map.detach().numpy()) / 1.0
  return map, max_val, mean_val, 1.0


def gen_saliency_map(X_train, Y_train,  
                     save_saliency_model = SAL_LOCATION, 
                     save_classifier = 'base_classifier_cifar'): 

    SAVE_CLASSIFIER = save_classifier
    SAVE_SAL_MODEL = save_saliency_model


    # get pretrained saliency model
    model = SimpleClassifier(base_channels=32, num_classes=10, image_channels=3)
    model.restore('base_classifier_cifar')

    # Default saliency model with pretrained resnet50 feature extractor, produces saliency maps which have resolution 4 times lower than the input image.
    sal_f = SaliencyModel(model, 3, 32, 3, 32, fix_encoder=True, use_simple_activation=False, allow_selector=True,  num_classes=NUM_CLASSES)
    sal_f.minimialistic_restore('sal_cifar_trial')

    #save stats and images
    maps=[]

    # switch axis as pytroch requires no. channels x height x width
    X_t= (np.moveaxis(X_train, 3, 1))
    Y_t = Y_train.astype(int)

    for k in range(len(X_train)):
        # iterate over images
        map, max_val, mean_val, sal_ind = generate_sal(sal_f, X_t[k], Y_t[k])
        temp_map = np.expand_dims(map.detach().numpy(), -1)
        maps.append(temp_map)

    return maps

In [0]:
def initiate_graph(input_shape, location, n_classes):
  #reset graph for the other model
    tf.reset_default_graph()
  
    #1: generate model 
    CNN_model = small_model(scope = "model",
                          nb_classes = n_classes,
                          model_params = [NB_FILTERS, N_HIDDEN_UNITS, 0.6, 0.6],
                          input_shape= input_shape)
    #load in model
    saver = tf.train.Saver() 
    sess= tf.Session()
    saver.restore(sess, location)
    
    return saver, sess, CNN_model
  
  
def compute_gradient(x_train, model, sess, x):
    X= tf.constant(x_train)
    Y = model.get_logits(X)
    g = tf.gradients(Y, [X])   
    
    return np.squeeze(sess.run(g, feed_dict={x:x_train}), axis=0)

In [0]:
def gen_adversarial_examples(sess, model, x_test):
  _, img_rows, img_cols, nchannels = x_test.shape
  nb_classes = 2
 # Define input placeholder
  x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                        nchannels))
  y = tf.placeholder(tf.float32, shape=(None, nb_classes))

  attack_params= {
        "method": "fgm",
        "eps": 0.005,
        "clip_min": 0.0,
        "clip_max": 1.0,
        "ord": np.inf,
    }
  adv_method = FastGradientMethod(model, sess=sess)  
  adv_x = adv_method.generate(x, **{k: a for k, 
                                      a in attack_params.items() if
                                      k != 'method'})

  # Evaluate the accuracy of the model on adversarial examples
  preds_adv = model.get_logits(adv_x)
  adv_images = adv_x.eval(session=sess, feed_dict={x: x_test})
  adv_preds = model.get_predicted_class(adv_x)
  adv_acc = adv_preds.eval(session = sess, feed_dict={x: x_test})
  return adv_images, adv_acc


In [0]:
#blurb: 
#location is an array containing the location of (1) the original classifier and (2) the detector
#x_train is the data (non-adversarial)
#y_train is a categorical vector containing true calsses
#data_method specifies the type of defense: images, sal_pixel, sal_map, concat_baseline
#rep determines how many iterations are performed
def ED(X):
  return np.expand_dims(X, -1)

def gen_attack(location = None, x_train =None, y_train = None, DATA_METHOD = None, rep=10):

  x_return =[]
  y_new = []
  n = len(x_train)
  adv_acc = np.zeros(n) 
  was_fooled = np.zeros(shape = (2*n,2))
  
  for j in range(rep):
    
    #Step 1: compute gradient of original classifier
    #Set-up model
    saver, sess, CNN_model= initiate_graph([BATCH_SIZE, 32, 32, 3], location[0],10)
    x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
    adv_preds = CNN_model.get_predicted_class(x)
    
    if j==0:
      prediction = adv_preds.eval(session = sess, feed_dict={x: x_train})
      a=0
      for j in range(len(x_train)):
        if prediction[j] ==y_c[j]:
          a+=1
      print("number of times original classifier is correct",a)

    x_new, adv_prediction = gen_adversarial_examples(sess, CNN_model, x_train)
    
    #compute the number of times it fools the original classifier 
    x_return =[]
    n = len(y_train)
    a=0
    for j in range(n):
      if adv_prediction[j] !=y_c[j]:
        was_fooled[j,0]=1
      if was_fooled[j,0] ==0: #if it was not previously fooled, update 
        x_return.append(x_new[j])
      else: #if it was previously fooled, do not update -> prevents reverting/saturating samples 
        x_return.append(x_train[j])
    print("number of times it fools image classifier", np.float32(sum(was_fooled[:,0]))/n)

    #generate defense data
    y_new=[]
      
    if DATA_METHOD =="sal_pixel" or DATA_METHOD == "sal_map":
      #compute saliency map
      sal_map_new = gen_saliency_map(x_return, adv_prediction,  
                     save_saliency_model = SAL_LOCATION, 
                     save_classifier = 'base_classifier')
      
      sal_map_orig = gen_saliency_map(x_train, y_train[:,1],  
                     save_saliency_model = SAL_LOCATION, 
                     save_classifier = 'base_classifier')
      
    if DATA_METHOD =="sal_pixel":
      x_new = x_return*np.repeat(sal_map_new, 3, -1)
      x_orig = x_train*np.repeat(sal_map_orig, 3, -1)
      y_new = np.concatenate((np.ones(len(x_new)), was_fooled[:n,0]), axis =0)
      y_new = K.utils.to_categorical(y_new)
      y_adv = np.concatenate((np.ones(len(x_new)), np.zeros(len(x_orig))), axis =0)
      x_train = np.concatenate((x_new, x_orig), axis=0)
    else:
      x_new = sal_map_new
      x_orig = sal_map_orig
      y_new = np.concatenate((was_fooled[:n,0], np.zeros(len(x_orig))), axis =0)
      y_new = K.utils.to_categorical(y_new)
      y_adv = np.concatenate((np.ones(len(x_new)), np.zeros(len(x_orig))), axis =0)
      x_train = np.concatenate((x_new, x_orig), axis=0)

    x_train = np.asarray(x_train)
    saver, sess, CNN_model= initiate_graph(X_shape_adv, location[1],2)
    x_new, adv_acc_c = gen_adversarial_examples(sess, CNN_model, x_train)
    perturbation = x_train[:n] - x_new[:n]
    
    for l in range(2*n):
      if adv_acc_c[l] !=np.where(y_new[l])[0]:
        was_fooled[l,1]=1
    for m in range(n):    
      if was_fooled[m,1] ==0: #if it was not previously fooled, update
        new_image = x_return[m]- perturbation[m]
        new_image = np.clip(new_image, a_min=0, a_max=1)
        x_return[m] = new_image
    print("no. of times it fools detector classifier", np.float32(sum(was_fooled[:,1]))/(2*n))
    
    if DATA_METHOD != "sal_map":
      x_train= np.asarray(x_return)[:n,:,:,:3]
    else:
      x_copy = x_train
      x_train = np.float32(np.asarray(x_return))
      x_return= np.expand_dims(x_copy[:,:,:,-1],-1)

  print('here')
  
  return np.float32(np.asarray(x_return)), y_new
  

In [0]:
#iteration epochs
for iter in range(10):
  #iteration through data sets 
  for k in range(N_STEPS):
    #read in data
    begin_index = MINI_BATCH*k
    end_index = begin_index +MINI_BATCH
    x_iter = x_train[begin_index: end_index]
    y_iter = y_train[begin_index: end_index]
    y_c = y_t[begin_index:end_index]
    
    print("attack starts")     
    x_adv, y_adv = gen_attack(location = locations, x_train =x_iter, y_train = y_iter, DATA_METHOD = DATA_METHOD, rep=100)

    print("defense starts")
    saver, sess, CNN_model= initiate_graph(X_shape_adv, locations[1], 2)
    optimizer = tf.train.AdamOptimizer(learning_rate = train_params['learning_rate'])
    report = AccuracyReport()
    set_log_level(logging.DEBUG)

    # Image Parameters
    _, img_rows, img_cols, nchannels = X_shape_adv
    
    # Define input placeholder
    x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                          nchannels))
    y = tf.placeholder(tf.float32, shape=(None, 2))  

    # Loss function
    loss = CrossEntropy(CNN_model, smoothing=0.0005)

    #Train
    for epch in range(10): #10
      train(sess, loss, x_adv, y_adv, optimizer = optimizer, 
      args=train_params, var_list=CNN_model.get_params(), loss_threshold =2e7) #uses Adam

    #Evaluation Phase 
    print("evaluation starts")
    y_var = [y_adv]
    preds = CNN_model.get_predicted_class(x)
    acc = preds.eval(session = sess, feed_dict={x: x_adv})
    for _, y_v in enumerate(y_var):
      #our attack 
      corr=0
      for f in range(len(acc)):
        if acc[f]== y_t[f]:
          corr+=1
      print(corr/len(y_v))

    if save_model:
      saver.save(sess, "./Checkpoints/model_adv_cifar_"+ DATA_METHOD + "_iter_" + str(iter) +"_" +str(k)+".ckpt")  
      print("model saved")

    locations[1]="./Checkpoints/model_adv_cifar_"+ DATA_METHOD + "_iter_" + str(iter) +"_" +str(k)+".ckpt"